## Inverse Distance Weighting (IDW) Interpolation

Let us suppose we have a data that shows the variation of one quantity of interest across space.
This could be equivalently viewed as { ($\vec{x_1}, y_1)$,$(\vec{x_2}, y_2)$,$(\vec{x_3}, y_3)$, ...}, where the $\vec{x_i}$'s represent the coordinates of the points where we have data and the $y_i$'s are the actual data at those points. <br><br>
We would like to perform an interpolation using these data points such that a few things are satisifed.
1. The interpolation is exact - the value at the known data points is the same as the estimated value, and 
2. We would want far away points from a given source data point to receive less importance than nearby points.
3. Wikipedia has an excellent article on IDW. I am linking it [here](https://en.wikipedia.org/wiki/Inverse_distance_weighting).

We are using the following approximation for coordinate_type being latlong_small<br>
$| \vec{r_2}− \vec{r_1}| ≈ \text{R }\times \sqrt[]{(Lat_2 - Lat_1)^{2} + (Long_2 - Long_1)^{2}}$

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('../test_data/30-03-18.csv')
data = np.array(df[['longitude','latitude','value']])

In [2]:
def make_grid(X,y,res):
    y_min = y.min()-0.2
    y_max = y.max()+0.2
    x_min = X.min()-0.2
    x_max = X.max()+0.2
    x_arr = np.linspace(x_min,x_max,res)
    y_arr = np.linspace(y_min,y_max,res)
    xx,yy = np.meshgrid(x_arr,y_arr)  
    return xx,yy

def idw(dataset, exponent = 2,  resolution='standard', coordinate_type='euclidean',verbose='False'):
    """
        Here X is the set of spatial locations - Usually assumed to be Lat-Long
        To be extended to higher dimenstions y - estimated value , exponenet - how
        much weight to assign to far off locations to be estimated for each data point, 
        extent - interpolate over a grid - what is xmax xmin ymax ymin
    """
    if coordinate_type == 'latlong_small':
        """
            Assume that the Earth is a Sphere, and use polar coordinates
            $| \vec{r_2}− \vec{r_1}| ≈ \text{R }\times \sqrt[]{(Lat_2 - Lat_1)^{2} + (Long_2 - Long_1)^{2}}$
        """
        return "To be done later"
    if coordinate_type == 'latlong_large':
        """
            Code to be written after understanding all the projections.
        """
        return "To be done later"
    if coordinate_type=="euclidean":
        
        print(dataset)
        X = dataset[:,0]
        y = dataset[:,1]
        if resolution=='high':
            xx,yy = make_grid(X,y,1000)
            
        if resolution=='low':
            xx,yy = make_grid(X,y,10)
            
        if resolution=='standard':
            xx,yy = make_grid(X,y,100)
        
        print(xx,yy)
        new = []
        new_arr = dataset
        for points in new_arr:
            mindist = np.inf
            val = 0
            for j in range(len(yy)):
                temp = yy[j][0]
                for i in range(len(xx[0])):
                    dist = np.linalg.norm(np.array([xx[0][i],temp]) - points[:2])
                    if dist<mindist:
                        mindist = dist
                        val = (i,j)
            new.append((points,val))
        new_grid = np.zeros((len(xx),len(yy)))
        for i in range(len(new)):
            x = new[i][1][0]
            y = new[i][1][1]
            new_grid[x][y] = new[i][0][2]
        x_nz,y_nz = np.nonzero(new_grid)
        list_nz = []
        for i in range(len(x_nz)):
            list_nz.append((x_nz[i],y_nz[i]))
        
        final = np.copy(new_grid)
        
        for i in range(len(xx[0])):
            for j in range(len(yy)):
                normalise = 0
                if (i,j) in list_nz:
                    continue
                else:
                    """
                    Could potentially have a divide by zero error here
                    Use a try except clause
                    """
                    for elem in range(len(x_nz)):
                        source = np.array([x_nz[elem],y_nz[elem]])
                        target = np.array([xx[0][i],yy[j][0]])
                        dist = (np.abs(xx[0][source[0]] - target[0])**exponent + np.abs(yy[source[1]][0] - target[1])**exponent)**(1/exponent)
                        final[i][j]+=new_grid[x_nz[elem],y_nz[elem]]/dist
                        normalise+=1/(dist)
                final[i][j]/=normalise
    
    return final


In [4]:
idw(data)

[[ 77.234291   28.581197  194.       ]
 [ 77.245721   28.739434  267.       ]
 [ 77.101961   28.822931  273.       ]
 [ 76.991463   28.620806  129.       ]
 [ 77.0325413  28.60909   176.       ]
 [ 77.072196   28.570859  172.       ]
 [ 77.1670103  28.5646102 168.       ]
 [ 77.1180053  28.5627763 105.       ]
 [ 77.272404   28.530782  203.       ]
 [ 77.26075    28.563827  192.       ]
 [ 77.0996943  28.610304   95.       ]
 [ 77.2273074  28.5918245 148.       ]
 [ 77.09211    28.732219  203.       ]
 [ 77.317084   28.668672  221.       ]
 [ 77.1585447  28.6573814 141.       ]
 [ 77.2011573  28.6802747 192.       ]
 [ 77.237372   28.612561  203.       ]
 [ 77.305651   28.632707  152.       ]
 [ 77.1473105  28.6514781 185.       ]
 [ 77.16482    28.699254  290.       ]
 [ 77.170221   28.728722  273.       ]
 [ 77.2005604  28.6372688 173.       ]
 [ 77.2011573  28.7256504 269.       ]
 [ 77.136777   28.669119  160.       ]
 [ 77.267246   28.49968    78.       ]
 [ 77.2494387  28.6316945

array([[177.13387494, 177.10444155, 177.0751776 , ..., 182.51067147,
        182.54423599, 182.57559186],
       [177.08625564, 177.05530927, 177.02448664, ..., 182.57854972,
        182.61138996, 182.64196579],
       [177.03843649, 177.00593458, 176.97350746, ..., 182.64894494,
        182.6809499 , 182.71063677],
       ...,
       [179.26995558, 179.29103787, 179.31416695, ..., 185.28492338,
        185.25669216, 185.22788513],
       [179.33494632, 179.35715441, 179.38137463, ..., 185.22148949,
        185.19510688, 185.16810927],
       [179.39792911, 179.42114488, 179.44633401, ..., 185.15891701,
        185.13430379, 185.10904101]])